有不同padding方式的game_of_live

In [0]:
import matplotlib.pyplot as plt
import numpy as np
plt.ion()

#iniliat the first matrix 
def creat_matrix(sizes):
    matrix = np.random.randint(0,2, size=(sizes,sizes))
    return matrix

'''
#padding 0
def Padding(matrix):
    padding_matrix = np.pad(matrix, 1, 'constant', constant_values = 0)
    return  padding_matrix
'''
#padding another side number 
def Padding(matrix):
    sizes = matrix.shape[0]
    new_padding_matrix = np.random.randint(0,1, size=(sizes*3,sizes*3))
    padding_matrix = np.pad(matrix, sizes, 'constant', constant_values = 0)
    new_padding_matrix = new_padding_matrix + padding_matrix
    new_padding_matrix = new_padding_matrix + np.roll(padding_matrix, sizes, axis=0) + np.roll(padding_matrix, -1*sizes, axis=0)
    new_padding_matrix = new_padding_matrix + np.roll(new_padding_matrix, sizes, axis=1) + np.roll(new_padding_matrix,-1*sizes, axis=1)
    padding_matrix = new_padding_matrix[sizes-1:sizes*2+1,sizes-1:sizes*2+1]
    return padding_matrix


#def the game
def Game_of_live(matrix):
    sizes = matrix.shape[0]
    result_matrix = np.random.randint(0,1, size=(sizes,sizes))
    padding_matrix = Padding(matrix)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            cross_matrix = padding_matrix[i:i+3, j:j+3] * test_matrix
            if (cross_matrix.sum() < 2):
                result_matrix[i,j] = 0
            elif (cross_matrix.sum() == 2 or cross_matrix.sum() == 3):
                result_matrix[i,j] = 1
            elif (cross_matrix.sum() > 3):
                result_matrix[i,j] = 0
    rtn = result_matrix
    return rtn
##########
import random
def Random_Matrix(row,column,start,finish):   
    matrix = [np.array(np.random.randint(start,finish, size=column)) for i in range(row)]
    return np.array(matrix)
##########

In [0]:
#印出game_of_live
########
if __name__ == '__main__': 
    test_matrix = np.array([[1, 1, 1],[1, 0, 1],[1, 1, 1]])
    matrix = creat_matrix(20) #input the size of square matrix
    for i in range (30):      #input the times of Game_of_live
        rnt = Game_of_live(matrix)
        matrix = rnt
        plt.imshow(matrix)
        plt.show()
##########

老師CNN範例

In [17]:
# 下面來定義 CNN 網路
from keras.models import Model
from keras.layers import Conv2D, Input

# 權重
def W(size,dtype):    
    rtn = np.ones(shape=(3,3,1,4))
    rtn[1,1,0,2:] = 10
    return rtn

def b(size,dtype):    
    return np.array([-2,-3, -12,-13])

def W2(size,dtype):
    return np.array([1,-2,1,-2]).reshape(1,1,4,1)

def b2(size,dtype):
    # just to be safe
    return np.full(size, -0.5)

# 網路模型定義
inputs = Input(shape=(None,None,1))
hidden = Conv2D(filters=4, kernel_size=3, padding='same', activation="relu",
             kernel_initializer=W, bias_initializer=b)(inputs)
out = Conv2D(filters=1, kernel_size=1, padding='same', activation="relu",
             kernel_initializer=W2, bias_initializer=b2)(hidden)
model = Model(inputs, out)

# 檢查看看結果是否正確
N = 10
# 隨機 100x100 盤面
boards = np.random.randint(0,2, size=(N,100,100))
# 用 CNN 模型跑下個盤面
rtn = model.predict(boards[..., None])
# >0 的值當成活著， <0 的值當成死的 (應該不會有 0的值)
rtn = (rtn>0).astype('int')
# 一一檢查
for i in range(N):
    b = game(boards[i])
    assert (b == rtn[i, :, :, 0]).all()
    print("OK", i)







OK 0
OK 1
OK 2
OK 3
OK 4
OK 5
OK 6
OK 7
OK 8
OK 9


嘗試改寫CNN權重,以random來猜測

In [41]:
##########
# 下面來定義 CNN 網路
from keras.models import Model
from keras.layers import Conv2D, Input

# 權重
def W(size,dtype):    
    rtn = np.ones(shape=(3,3,1,4))
    rtn[1,1,0,2:] = random.randint(0,10)
    return rtn

def b(size,dtype):
    return Random_Matrix(1,4,-12,0).reshape(4)   
    #return np.array([-2,-9,-3,-4])

def W2(size,dtype):
    return Random_Matrix(1,4,-2,2).reshape(1,1,4,1)
    #return np.array([1,-2,1,-2]).reshape(1,1,4,1)

def b2(size,dtype):
    # just to be safe
    bias2 = np.random.randn()
    return np.full(size,bias2)
    #return np.full(size, -0.5)

N = 10
boards = np.random.randint(0,2, size=(N,3,3))    
times=100
best_accuracy = 0
for i in range(times):
    count = 0
    # 網路模型定義
    inputs = Input(shape=(None,None,1))
    hidden = Conv2D(filters=4, kernel_size=3, padding='same', activation="relu",
                 kernel_initializer=W, bias_initializer=b)(inputs)
    out = Conv2D(filters=1, kernel_size=1, padding='same', activation="relu",
                 kernel_initializer=W2, bias_initializer=b2)(hidden)
    model = Model(inputs, out)
    
    bias = model.get_weights()[1]
    bias2 = model.get_weights()[3]
    weight = model.get_weights()[0][1,1,0,2:]
    weight2 = model.get_weights()[2].reshape(4)
    
    ######
    rtn = model.predict(boards[..., None])
    rtn = (rtn>0).astype('int')
    # 一一檢查
    for j in range(N):
        Game_answer = Game_of_live(boards[j])
        if ((Game_answer == rtn[j, :, :, 0]).all()):
            count +=1
        else:
            pass;
    if ((count/N)>=best_accuracy):
        print("第{}次測試,Accuracy:{}".format(i,count/N))
        best_accuracy = count/N 
        print("bias",bias)
        print("bias2",bias2)
        print("weight",weight)
        print("weight2",weight2)

第0次測試,Accuracy:0.3
bias [-8. -9. -8. -5.]
bias2 [0.74744475]
weight [1. 1.]
weight2 [-1.  1. -1.  1.]
第1次測試,Accuracy:0.3
bias [-7. -9. -5. -1.]
bias2 [1.9944521]
weight [10. 10.]
weight2 [-1. -1.  1.  0.]
第3次測試,Accuracy:0.4
bias [-6. -4. -5. -9.]
bias2 [-0.01461779]
weight [0. 0.]
weight2 [ 1.  0. -1. -2.]
第4次測試,Accuracy:0.4
bias [ -6.  -1.  -8. -11.]
bias2 [-0.43277276]
weight [6. 6.]
weight2 [0. 0. 0. 1.]
第5次測試,Accuracy:0.4
bias [-3. -5. -8. -5.]
bias2 [-0.8542179]
weight [4. 4.]
weight2 [-2. -1.  1. -2.]
第6次測試,Accuracy:0.4
bias [ -3. -12.  -1.  -6.]
bias2 [-1.579965]
weight [1. 1.]
weight2 [-2. -2.  0. -2.]
第7次測試,Accuracy:0.4
bias [-12. -10.  -7.  -9.]
bias2 [-1.1133618]
weight [9. 9.]
weight2 [ 0.  1. -2. -2.]
第8次測試,Accuracy:0.4
bias [-12.  -8.  -7.  -6.]
bias2 [-0.16515389]
weight [6. 6.]
weight2 [ 0. -1.  1. -2.]
第9次測試,Accuracy:0.4
bias [-12.  -7.  -5. -11.]
bias2 [-2.7255917]
weight [3. 3.]
weight2 [ 1. -1. -1. -1.]
第11次測試,Accuracy:0.4
bias [ -7. -11.  -5. -11.]
bias2 [-0.006435

效果很差手動輸入

In [27]:
from keras.models import Model
from keras.layers import Conv2D, Input
def W(size,dtype):    
    rtn = np.ones(shape=(3,3,1,4))
    rtn[1,1,0,2:] = 9
    return rtn
def b(size,dtype):
    return np.array([-2,-3, -11,-12])
def W2(size,dtype):
    return np.array([1,-2,1,-2]).reshape(1,1,4,1)
def b2(size,dtype):
    return np.full(size, -0.5)
inputs = Input(shape=(None,None,1))
hidden = Conv2D(filters=4, kernel_size=3, padding='same', activation="relu",
             kernel_initializer=W, bias_initializer=b)(inputs)
out = Conv2D(filters=1, kernel_size=1, padding='same', activation="relu",
             kernel_initializer=W2, bias_initializer=b2)(hidden)
model = Model(inputs, out)

bias = model.get_weights()[1]
print(bias)
bias2 = model.get_weights()[3]
print(bias2)

N = 10

boards = np.random.randint(0,2, size=(N,5,5))

rtn = model.predict(boards[..., None])

rtn = (rtn>0).astype('int')

for i in range(N):
    b = game(boards[i])
    #print(b)
    #print(rtn[i, :, :, 0])
    print("-----------")
    assert (b == rtn[i, :, :, 0]).all()
    print("OK", i)    

[ -2.  -3. -11. -12.]
[-0.5]
-----------
OK 0
-----------
OK 1
-----------
OK 2
-----------
OK 3
-----------
OK 4
-----------
OK 5
-----------
OK 6
-----------
OK 7
-----------
OK 8
-----------
OK 9
